<a href="https://colab.research.google.com/github/medha-sagar/CSE547-BigDataForMachineLearning/blob/master/CSE547_Q1_PeopleYouMightKnow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 217.8MB 69kB/s 
     |████████████████████████████████| 204kB 61.0MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=01d1ad60317e1f0200dea232a06000a53a7de1b75d77c8bfca6062bf92b2cd63
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark
openjdk-8-jdk-headless is already the newest version (8u242-b08-0ubuntu3~18.04).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
id='19i-TnVx8qjX7q8XKdR59UbiA5_7Z5qLU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('soc-LiveJournal1Adj.txt')

In [0]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [8]:
friend_data = spark.createDataFrame(
    sc.textFile("soc-LiveJournal1Adj.txt").map(lambda l: l.split('\t')),
    ["person","friends"]
)
friend_data.show(5)

+------+--------------------+
|person|             friends|
+------+--------------------+
|     0|1,2,3,4,5,6,7,8,9...|
|     1|0,5,20,135,2409,8...|
|     2|0,117,135,1220,27...|
|     3|0,12,41,55,1532,1...|
|     4|0,8,14,15,18,27,7...|
+------+--------------------+
only showing top 5 rows



In [9]:
person_friend = friend_data.withColumn("friends", explode(split("friends", "[,]")) )
person_friend.sort(person_friend.person).show(5)

+------+-------+
|person|friends|
+------+-------+
|     0|      1|
|     0|      2|
|     0|      3|
|     0|      4|
|     0|      5|
+------+-------+
only showing top 5 rows



In [11]:
person_friend.registerTempTable("PersonFriend")

query ="""
SELECT a.person, a.friends, b.friends as friend_of_friend
FROM PersonFriend a INNER JOIN PersonFriend b
ON b.person = a.friends
GROUP BY a.person, a.friends, b.friends
ORDER BY a.person, a.friends
"""

friend_of_a_friend = spark.sql(query)
friend_of_a_friend.show(10)

+------+-------+----------------+
|person|friends|friend_of_friend|
+------+-------+----------------+
|     0|      1|           12846|
|     0|      1|            8715|
|     0|      1|           12347|
|     0|      1|              20|
|     0|      1|            2409|
|     0|      1|            8932|
|     0|      1|           10623|
|     0|      1|               0|
|     0|      1|               5|
|     0|      1|             135|
+------+-------+----------------+
only showing top 10 rows



In [13]:
friend_of_a_friend.registerTempTable("MutualFriends")

query = """
SELECT person, count(friends) as num_mutual_friends, friend_of_friend
FROM MutualFriends
WHERE person != friend_of_friend
GROUP BY person, friend_of_friend
ORDER BY person, friend_of_friend, count(friends)
"""

no_mutual_friends = spark.sql(query)
no_mutual_friends.show()

+------+------------------+----------------+
|person|num_mutual_friends|friend_of_friend|
+------+------------------+----------------+
|     0|                 2|               1|
|     0|                 3|              10|
|     0|                 1|           10001|
|     0|                 1|            1001|
|     0|                 1|           10014|
|     0|                 1|           10018|
|     0|                 1|           10020|
|     0|                 1|           10023|
|     0|                 1|           10025|
|     0|                 1|           10038|
|     0|                 1|           10041|
|     0|                 1|           10042|
|     0|                 1|           10045|
|     0|                 1|           10048|
|     0|                 1|           10055|
|     0|                 1|           10058|
|     0|                 1|           10059|
|     0|                 1|           10063|
|     0|                 1|           10066|
|     0|  

In [16]:
no_mutual_friends.registerTempTable("NumMutualFriends")

query = """
SELECT a.person, a.friend_of_friend, a.num_mutual_friends
FROM NumMutualFriends a
WHERE a.friend_of_friend not in (SELECT friends FROM PersonFriend f where f.person = a.person)
ORDER BY a.person, a.friend_of_friend, a.num_mutual_friends desc
"""

mutual_friends = spark.sql(query)
mutual_friends.show()

+------+----------------+------------------+
|person|friend_of_friend|num_mutual_friends|
+------+----------------+------------------+
|     0|           10001|                 1|
|     0|            1001|                 1|
|     0|           10014|                 1|
|     0|           10018|                 1|
|     0|           10020|                 1|
|     0|           10023|                 1|
|     0|           10025|                 1|
|     0|           10038|                 1|
|     0|           10041|                 1|
|     0|           10042|                 1|
|     0|           10045|                 1|
|     0|           10048|                 1|
|     0|           10055|                 1|
|     0|           10058|                 1|
|     0|           10059|                 1|
|     0|           10063|                 1|
|     0|           10066|                 1|
|     0|           10077|                 1|
|     0|           10086|                 1|
|     0|  

In [18]:
mutual_friends.filter(mutual_friends.person==11).orderBy(mutual_friends.num_mutual_friends.desc(), mutual_friends.friend_of_friend).show()

+------+----------------+------------------+
|person|friend_of_friend|num_mutual_friends|
+------+----------------+------------------+
|    11|           27552|                 4|
|    11|           27573|                 3|
|    11|           27574|                 3|
|    11|           27589|                 3|
|    11|           27590|                 3|
|    11|           27600|                 3|
|    11|           27617|                 3|
|    11|           27620|                 3|
|    11|           27667|                 3|
|    11|           32072|                 3|
|    11|           33192|                 3|
|    11|            7785|                 3|
|    11|              10|                 2|
|    11|           10253|                 2|
|    11|           10782|                 2|
|    11|             110|                 2|
|    11|              12|                 2|
|    11|           12666|                 2|
|    11|            1797|                 2|
|    11|  

In [24]:
mutual_friends.filter(mutual_friends.person==924).sort(mutual_friends.num_mutual_friends.desc()).sort(mutual_friends.friend_of_friend).show()
#439, 2409, 6995, 11860, 15416, 43748, 43748

+------+----------------+------------------+
|person|friend_of_friend|num_mutual_friends|
+------+----------------+------------------+
|   924|           11860|                 1|
|   924|           15416|                 1|
|   924|            2409|                 1|
|   924|           43748|                 1|
|   924|             439|                 1|
|   924|           45881|                 1|
|   924|            6995|                 1|
+------+----------------+------------------+



In [26]:
mutual_friends.filter(mutual_friends.person==8941).sort(mutual_friends.num_mutual_friends.desc(), mutual_friends.friend_of_friend).show()
# 8943 8944 8940

+------+----------------+------------------+
|person|friend_of_friend|num_mutual_friends|
+------+----------------+------------------+
|  8941|            8943|                 2|
|  8941|            8944|                 2|
|  8941|            8940|                 1|
+------+----------------+------------------+



In [27]:
mutual_friends.filter(mutual_friends.person==8942).sort(mutual_friends.num_mutual_friends.desc(), mutual_friends.friend_of_friend).show()
# 8939 8940 8943 8944

+------+----------------+------------------+
|person|friend_of_friend|num_mutual_friends|
+------+----------------+------------------+
|  8942|            8939|                 3|
|  8942|            8940|                 1|
|  8942|            8943|                 1|
|  8942|            8944|                 1|
+------+----------------+------------------+



In [30]:
mutual_friends.filter(mutual_friends.person==9019).sort(mutual_friends.num_mutual_friends.desc(), mutual_friends.friend_of_friend).show()
# 9022 317 9023

+------+----------------+------------------+
|person|friend_of_friend|num_mutual_friends|
+------+----------------+------------------+
|  9019|            9022|                 2|
|  9019|             317|                 1|
|  9019|            9023|                 1|
+------+----------------+------------------+



In [31]:
#9020
mutual_friends.filter(mutual_friends.person==9020).sort(mutual_friends.num_mutual_friends.desc(), mutual_friends.friend_of_friend).show()
# 9021 9016 9017 9022 317 9023

+------+----------------+------------------+
|person|friend_of_friend|num_mutual_friends|
+------+----------------+------------------+
|  9020|            9021|                 3|
|  9020|            9016|                 2|
|  9020|            9017|                 2|
|  9020|            9022|                 2|
|  9020|             317|                 1|
|  9020|            9023|                 1|
+------+----------------+------------------+



In [34]:
#9021
mutual_friends.filter(mutual_friends.person==9021).sort(mutual_friends.num_mutual_friends.desc(), mutual_friends.friend_of_friend).show()
# 9020 9016 9017 9022 317 9023

+------+----------------+------------------+
|person|friend_of_friend|num_mutual_friends|
+------+----------------+------------------+
|  9021|            9020|                 3|
|  9021|            9016|                 2|
|  9021|            9017|                 2|
|  9021|            9022|                 2|
|  9021|             317|                 1|
|  9021|            9023|                 1|
+------+----------------+------------------+



In [36]:
#9022
mutual_friends.filter(mutual_friends.person==9022).sort(mutual_friends.num_mutual_friends.desc(), mutual_friends.friend_of_friend).show()
#9019 9020 9021 317 9016 9017 9023

+------+----------------+------------------+
|person|friend_of_friend|num_mutual_friends|
+------+----------------+------------------+
|  9022|            9019|                 2|
|  9022|            9020|                 2|
|  9022|            9021|                 2|
|  9022|             317|                 1|
|  9022|            9016|                 1|
|  9022|            9017|                 1|
|  9022|            9023|                 1|
+------+----------------+------------------+



In [38]:
#9990
mutual_friends.filter(mutual_friends.person==9990).sort(mutual_friends.num_mutual_friends.desc(), mutual_friends.friend_of_friend).show()
# 13134 13478 13877 34299 34485 34642 37941

+------+----------------+------------------+
|person|friend_of_friend|num_mutual_friends|
+------+----------------+------------------+
|  9990|           13134|                 1|
|  9990|           13478|                 1|
|  9990|           13877|                 1|
|  9990|           34299|                 1|
|  9990|           34485|                 1|
|  9990|           34642|                 1|
|  9990|           37941|                 1|
+------+----------------+------------------+



In [32]:
#9992
mutual_friends.filter(mutual_friends.person==9992).sort(mutual_friends.num_mutual_friends.desc(), mutual_friends.friend_of_friend).show()
# 9987 9989 35667 9991

+------+----------------+------------------+
|person|friend_of_friend|num_mutual_friends|
+------+----------------+------------------+
|  9992|            9987|                 4|
|  9992|            9989|                 4|
|  9992|           35667|                 3|
|  9992|            9991|                 2|
+------+----------------+------------------+



In [33]:
#9993
mutual_friends.filter(mutual_friends.person==9993).sort(mutual_friends.num_mutual_friends.desc(), mutual_friends.friend_of_friend).show()
# 9991 13134 13478 34299 34485 34642 37941

+------+----------------+------------------+
|person|friend_of_friend|num_mutual_friends|
+------+----------------+------------------+
|  9993|            9991|                 5|
|  9993|           13134|                 1|
|  9993|           13478|                 1|
|  9993|           13877|                 1|
|  9993|           34299|                 1|
|  9993|           34485|                 1|
|  9993|           34642|                 1|
|  9993|           37941|                 1|
+------+----------------+------------------+

